In [23]:
# this is boiler plate for when we move stuff into SQL. I think it will need to be cleaned before SQL. 

import pandas as pd
import pyodbc
import time
import numpy as np
from datetime import datetime, date, time, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)


In [24]:
surveyMonkey_raw = pd.read_csv('Family Futures EventTabling Feedback.csv')
#surveyMonkey_raw = pd.read_csv('Family Futures EventTabling Feedback.csv', encoding='utf-8', quoting=1)

In [25]:
surveyMonkey_raw = surveyMonkey_raw.iloc[1:].reset_index(drop=True)

In [26]:
# to get the names of the columns that need to be renamed
surveyMonkey_raw.columns



Index(['Respondent ID', 'Collector ID', 'Start Date', 'End Date', 'IP Address',
       'Email Address', 'First Name', 'Last Name', 'Custom Data 1',
       'What is your full name?', 'Unnamed: 10', 'Unnamed: 11',
       'What was your role at this event/tabling?', 'Unnamed: 13',
       'Are you providing feedback from a Community Event or from Partner Tabling?\n\nA Community Event is a special occasion (ex. Summer Celebration Bonanza at the Grand Rapids Public Library)\nPartner Tabling is when you spend time tabling at a partner organization without there being a special occasion, celebration, etc. (ex. Tabling at the library on the last Friday of the month)\n',
       'What event did you attend?If the event you attended is not in this list, click here to complete a data/tech request. The item will be added shortly and you will be notified when it is done.',
       'What was the date of the event?',
       'Were you the primary lead/assignee for this event?',
       'What organization w

In [27]:
# This cell needs to be checked everytime there are changes to the survey monkey! Pay close attention to the
#  "Unnamed" those skip around often, if one chanes you have to check them all.

columns_to_rename = {

    'Custom Data 1': 'xxxx',
    'Email':'x',
    'First Name':'xx',
    'Last Name':'xxx',
    'What is your full name?':'First Name',
    'Unnamed: 10':'Last Name',
    'Unnamed: 11':'Email',
    'Unnamed: 28':'Clicker_Other',
    'Please provide the number of people engaged at the Changing Station / JWNN (leave blank if not applicable):': 'Nursing',
    'Unnamed: 24':'Changing',
    'Unnamed: 25':'NN Adult Engaged',
    'Unnamed: 26':'NN Kids Engaged',
    'Unnamed: 27':'Both',
    'What is your Favorite Color?':'Name_OLD',
    'Are you providing feedback from a Community Event or from Partner Tabling?\n\nA Community Event is a special occasion (ex. Summer Celebration Bonanza at the Grand Rapids Public Library)\nPartner Tabling is when you spend time tabling at a partner organization without there being a special occasion, celebration, etc. (ex. Tabling at the library on the last Friday of the month)\n': 'Event/Tabling',
    'What was your role at this event/tabling?': 'Role',
    'Unnamed: 12': 'Role_Other',
    'What event did you attend?If the event you attended is not in this list, click here to complete a data/tech request. The item will be added shortly and you will be notified when it is done.': 'Name (Event)',
    'What was the date of the event?': 'Date',
    'Were you the primary lead/assignee for this event?': 'Lead Assignee Event',
    'What organization were you tabling at?If the organization you tabled at is not in this list, click here to complete a data/tech request. The item will be added shortly and you will be notified when it is done.': 'Tabling Location',
    'Please enter the date that you were tabling at this location:': 'Tabling Date',
    'Were you the primary lead/assignee for this tabling?': 'Lead Assignee (Tabling)',
    'Please enter the number of kids engaged with at this event/tabling.': 'Number Kids Engaged',
    'Please enter the number of adults engaged with at this event/tabling.': 'Number Adults Engaged',
    'Were the above numbers (kids and adults engaged with) counted using a clicker or are they your best estimate?': 'Clicker?',
    'Unnamed: 29': 'Clicker Other',
    'Did you network with any organizations? (aside from the organization that was hosting the event or tabling)': 'Networked Orgs?',
    'If yes, list the name(s) of the organization(s) you networked with here (please write the full name, not the acronym):': 'Names of Networked Orgs',
    'If applicable, what activities/materials did we bring to this event or tabling? What was your experience with them? (ex. Were things easy or hard to use? Did families engage with them? etc.)': 'Activities/Materials Feedback',
    'Would you recommend attending this event or tabling at this location again?': 'Recommend Again?',
    'Why or why not?': 'Recommendation Reason',
    'Please add a "mission moment" or story/anecdote in the space below.': 'Mission Moment',
    'Please add any further comments below.': 'Further Comments'
}



In [28]:
surveyMonkey_raw.rename(columns=columns_to_rename,inplace = True)

In [29]:
surveyMonkey_raw['Date'] = pd.to_datetime(surveyMonkey_raw['Date'], errors='coerce').dt.date





for i in range(len(surveyMonkey_raw)):
    if surveyMonkey_raw.loc[i, 'Event/Tabling'] == 'Community Event':
        surveyMonkey_raw.loc[i, 'Event Date'] = surveyMonkey_raw.loc[i, 'Date']
        surveyMonkey_raw.loc[i, 'Event Name'] = surveyMonkey_raw.loc[i, 'Name (Event)']
        surveyMonkey_raw.loc[i, 'Lead Assignee'] = surveyMonkey_raw.loc[i, 'Lead Assignee Event']
    elif surveyMonkey_raw.loc[i, 'Event/Tabling'] == 'Partner Tabling':
        surveyMonkey_raw.loc[i, 'Event Date'] = surveyMonkey_raw.loc[i, 'Tabling Date']
        surveyMonkey_raw.loc[i, 'Event Name'] = surveyMonkey_raw.loc[i, 'Tabling Location']
        surveyMonkey_raw.loc[i, 'Lead Assignee'] = surveyMonkey_raw.loc[i, 'Lead Assignee (Tabling)']
    else:
        surveyMonkey_raw.loc[i, 'Event Date'] = 'error'
        surveyMonkey_raw.loc[i, 'Event Name'] = 'error'
        surveyMonkey_raw.loc[i, 'Lead Assignee'] = 'error'
        print(f"Warning: Row {i} unexpected 'Event/Tabling' value: {surveyMonkey_raw.loc[i, 'Event/Tabling']}")
    
    
    
    
    

In [30]:
#The events data and the tabling data are kept in separate columns, the following code merges them.
surveyMonkey_raw = surveyMonkey_raw.copy()
surveyMonkey_raw = surveyMonkey_raw.reset_index(drop=True)
for i in range(len(surveyMonkey_raw)):
    if surveyMonkey_raw.iloc[i]['Event/Tabling'] == 'Community Event':
        surveyMonkey_raw.at[i, 'Event Date'] = surveyMonkey_raw.iloc[i]['Date']
        surveyMonkey_raw.at[i, 'Event Name'] = surveyMonkey_raw.iloc[i]['Name (Event)']
        surveyMonkey_raw.at[i, 'Lead Assignee'] = surveyMonkey_raw.iloc[i]['Lead Assignee Event']
    elif surveyMonkey_raw.iloc[i]['Event/Tabling'] == 'Partner Tabling':
        surveyMonkey_raw.at[i, 'Event Date'] = surveyMonkey_raw.iloc[i]['Tabling Date']
        surveyMonkey_raw.at[i, 'Event Name'] = surveyMonkey_raw.iloc[i]['Tabling Location']
        surveyMonkey_raw.at[i, 'Lead Assignee'] = surveyMonkey_raw.iloc[i]['Lead Assignee (Tabling)']
    else:
        surveyMonkey_raw.at[i, 'Event Date'] = 'error'
        surveyMonkey_raw.at[i, 'Event Name'] = 'error'
        surveyMonkey_raw.at[i, 'Lead Assignee'] = 'error'
        print(f"Warning: Row {i} unexpected 'Event/Tabling' value: {surveyMonkey_raw.iloc[i]['Event/Tabling']}")

In [31]:
print(surveyMonkey_raw.index.duplicated().sum()) 

0


In [32]:
#surveyMonkey_raw

In [33]:
#Need to combine Name_old and First Name + last Name into the Column Person Name for the next step in the data prep Survey Completion
#   This is because of the change in how name was collected after we had already done a large amount of events. 
#   Plan to pull this in 2026 when we do an offical update/refresh from scratch of the Survey.


# Start with Person Name as the combination of First and Last Name
surveyMonkey_raw['Person Name'] = (
    surveyMonkey_raw['First Name'].astype(str).str.strip() + ' ' +
    surveyMonkey_raw['Last Name'].astype(str).str.strip()
)

# For any rows where First Name is missing, replace Person Name with Name_OLD
mask = surveyMonkey_raw['First Name'].isna()
surveyMonkey_raw.loc[mask, 'Person Name'] = surveyMonkey_raw.loc[mask, 'Name_OLD']



In [34]:
Coumns_to_Remove = {    'Respondent ID','xx', 'xxx', 'xxxx',
    'Collector ID',
    'Start Date',
    'End Date',
    'Date',
    'Lead Assignee Event',
    'Tabling Location',
    'Tabling Date',
    'Lead Assignee (Tabling)',
    'IP Address',
    'Email Address',
    'Name (Event)', 
    'Unnamed: 13',
    'Name_OLD'       }

surveyMonkey_raw.drop(Coumns_to_Remove,inplace = True, axis = 1)



In [35]:
surveyMonkey_raw.head(45)

,First Name,Last Name,Email,Role,Event/Tabling,Number Kids Engaged,Number Adults Engaged,Nursing,Changing,NN Adult Engaged,NN Kids Engaged,Both,Clicker?,Clicker Other,Networked Orgs?,Names of Networked Orgs,Activities/Materials Feedback,Recommend Again?,Recommendation Reason,Mission Moment,Further Comments,Event Date,Event Name,Lead Assignee,Person Name
0,JD,Daniels,jdaniels@familyfutures.org,Family Futures Staff Member,Community Event,104,54,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,Yes,Friend of the Court,"Coloring books, crayons, sunglasses, balance b...",Yes,There were far more people than I expected. I ...,We were sat right next to our good friends in ...,NaN,2025-08-16,Taylor Made Re-entry Kickball Event,Yes,JD Daniels
1,Leticia,Miranda,lmiranda@familyfutures.org,Family Futures Staff Member,Community Event,180,101,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,No,NaN,Coloring books and crayon are a big hit,Yes,The time I table was in the morning from 9-12 ...,NaN,NaN,2025-08-14,SHOES Program,Yes,Leticia Miranda
2,Teya Li,Weber,Tweber@familyfutures.org,Intern,Community Event,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,"Balance boards, flyers, coloring books, crayon...",Yes,This was a big event with lots of families wit...,"We got to tell a kid about our organization, w...",NaN,2025-08-16,Taylor Made Re-entry Kickball Event,No,Teya Li Weber
3,Sandy,Portko,sportko@familyfutures.org,Family Futures Staff Member,Community Event,255,142,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,Yes,Downtown YMCA,Information about Family Futures: Connections ...,Yes,Great opportunity to increase community awaren...,I talked with several moms who were currently ...,"If we were to consider an activity, it would r...",2025-08-14,SHOES Program,Yes,Sandy Portko
4,Sandy,Portko,sportko@familyfutures.org,Family Futures Staff Member,Community Event,0,23,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,No,NaN,NaN,Yes,This was a talk/discussion that is part of the...,All of the parents were eager for information ...,The Telamon center is having a book drive for ...,2025-08-13,Kent City Migrant Head Start Parenting Talk,Yes,Sandy Portko
5,Susan,Alonso,salonso@familyfutures.org,Family Futures Staff Member,Community Event,30,16,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,Yes,Literacy Center of West Michigan,Stickers,Yes,Access to many families in the promise neighbo...,I saw a former navigator that worked for FF a ...,NaN,2025-08-13,Burton Open House and Orientation,Yes,Susan Alonso
6,Susan,Alonso,salonso@familyfutures.org,Family Futures Staff Member,Partner Tabling,23,24,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,No,NaN,"Standard tabling fare, coloring books",Yes,Access to diverse families,I met a family from Eritrea that lived in a re...,NaN,08/11/2025,Sheldon WIC Clinic,Yes,Susan Alonso
7,Lisa,Brown,lbrown@familyfutures.org,Family Futures Staff Member,Community Event,342,189,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,Yes,Luke at Project Green,"coloring books, crayons, bags, flyers, pens",Yes,Connects with MANY families in the community.,Several families telling me they went through ...,NaN,2025-08-13,SHOES Program,Yes,Lisa Brown
8,Lisa,Brown,lbrown@familyfutures.org,Family Futures Staff Member,Community Event,217,105,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-12,Backpacks and Beats,Yes,Lisa Brown
9,Leticia,Miranda,lmiranda@familyfutures.org,Family Futures Staff Member,Partner Tabling,2,6,NaN,NaN,NaN,NaN,NaN,I used a clicker,NaN,No,NaN,sunglasses,Yes,"no enrollments, it was a slow day",NaN,NaN,08/13/2025,HELP Pregnancy Aid,Yes,Leticia Miranda


In [36]:
# 1. Read in the cleaned date library
asana_dates = pd.read_csv('asana_dates_clean.csv')

# 2. Normalize spacing and casing just to be safe
#surveyMonkey_raw['Event Name'] = surveyMonkey_raw['Event Name'].str.strip().str.replace(r'\s+', ' ', regex=True)
#asana_dates['Name'] = asana_dates['Name'].str.strip().str.replace(r'\s+', ' ', regex=True)

# 3. Merge to pull correct Event Date
surveyMonkey_raw = surveyMonkey_raw.merge(
    asana_dates.rename(columns={'Name': 'Event Name', 'Date': 'Correct Event Date'}),
    on='Event Name',
    how='left'
)

# 4. Overwrite Event Date where we have a corrected match
surveyMonkey_raw['Event Date'] = surveyMonkey_raw['Correct Event Date'].combine_first(surveyMonkey_raw['Event Date'])

# 5. Drop helper column if needed
surveyMonkey_raw.drop(columns=['Correct Event Date'], inplace=True)

In [37]:
surveyMonkey_raw.to_csv('Local_surveyMonkey.csv',index = False)

In [38]:
# This section mkaes an unpacked copy of the data specifically for Tabluea engagement counts table. 

In [39]:


# Make a copy
survey_filtered = surveyMonkey_raw.copy()

# List of columns to check for non-null values
cols_to_check = [
    'Number Kids Engaged',
    'Number Adults Engaged',
    'Nursing',
    'Changing',
    'NN Adult Engaged',
    'NN Kids Engaged'
]

# Filter to keep only rows where at least one of these columns is not null or not empty
survey_filtered = survey_filtered[survey_filtered[cols_to_check].notna().any(axis=1)]

In [40]:
survey_filtered.columns

Coumns_to_Remove = { 'First Name', 'Last Name', 'Email', 'Role',
        'Clicker?', 'Clicker Other',
       'Networked Orgs?', 'Names of Networked Orgs',
       'Activities/Materials Feedback', 'Recommend Again?', 'Recommendation Reason',
                    'Mission Moment', 'Further Comments','Lead Assignee', 'Person Name'      }

survey_filtered.drop(Coumns_to_Remove,inplace = True, axis = 1)

In [41]:
# Sort by 'Event Date'
# Convert Event Date column to datetime
survey_filtered['Event Date'] = pd.to_datetime(survey_filtered['Event Date'], errors='coerce')
survey_filtered = survey_filtered.sort_values(by='Event Date')

# add event ID
survey_filtered['Event ID'] = ['E' + str(i + 1) for i in range(len(survey_filtered))]
survey_filtered['Event ID'] = survey_filtered['Event ID'].astype(str)

In [42]:
survey_filtered.columns

Index(['Event/Tabling', 'Number Kids Engaged', 'Number Adults Engaged',
       'Nursing', 'Changing', 'NN Adult Engaged', 'NN Kids Engaged', 'Both',
       'Event Date', 'Event Name', 'Event ID'],
      dtype='object')

In [43]:
import pandas as pd

# Columns to count from
count_cols = [
    'Number Kids Engaged',
    'Number Adults Engaged',
    'Nursing',
    'Changing',
    'NN Adult Engaged',
    'NN Kids Engaged'
]

# Columns to keep
keep_cols = ['Event/Tabling', 'Event Date', 'Event Name', 'Event ID']

# New DataFrame to store results
duplicated_rows = []

for idx, row in survey_filtered.iterrows():
    for col in count_cols:
        count = row[col]
        if pd.notna(count) and int(count) > 0:
            for _ in range(int(count)):
                duplicated_rows.append({
                    'Event/Tabling': row['Event/Tabling'],
                    'Event Date': row['Event Date'],
                    'Event Name': row['Event Name'],
                    'Event ID': row['Event ID'],
                    'Engagement Type': col
                })

# Now this will work cleanly
engagement_long = pd.DataFrame(duplicated_rows)

In [44]:
engagement_long.to_csv('Engagment_counts.csv')